In [2]:
!vehicle verify --specification vehicle/properties.vcl --verifier Marabou --network classifier:adv_mid_bs32_0.999_e8_adv.onnx --dataset trainingInputs:vehicle/trainingInputs.idx --property property > vehicle/outputs/adv_mid_bs32_0.999_e8_adv.txt

In [2]:
import idx2numpy

def get_valid_inputs(data, n, save=False):
    valid_inputs = []

    for point in data:
        valid_input = True
        valid_tcp = False
        valid_http = False
        valid_time_elapsed = False

        for element in point:
            if element < 0.0 or element > 1.0:
                valid_input = False
                break
            
        # Flag, Size, Direction, Protocol
        if point[7] == 2/256 and point[8] == 18/256 and point[9] == 16/256 and \
           point[17] >= 52/1000 and point[17] <= 60/1000 and point[18] >= 52/1000 and point[18] <= 60/1000 and point[19] >= 40/1000 and point[19] <= 52/1000 and \
           point[2] == 0 and point[3] == 1 and point[4] == 0 and \
           point[1] == 0:
            valid_tcp = True

        # Flag, Size, Direction, Protocol
        if point[10] == 24/256 and \
           point[20] >= 0/1000 and point[20] <= 500/1000 and \
           point[5] == 0:
            valid_http = True

        # TimeElapsed
        if point[0] == 0 or point[0] >= 0.1:
            valid_time_elapsed = True

        if valid_input and valid_tcp and valid_http and valid_time_elapsed:
            valid_inputs.append(point)
        
        if len(valid_inputs) >= n:
            break

        if save:
            idx2numpy.convert_to_file('./vehicle/trainingInputs.idx', valid_inputs)
      
    return valid_inputs

In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import os


pgd_steps = 5
batch_size = 32

train_data_df = pd.read_csv(f'./data/full-preprocessed-ssh-data-train.csv')
train_pos = np.array(train_data_df.loc[train_data_df.iloc[:, -1] == 0].iloc[:, :(len(train_data_df.columns)-1)].values.tolist())
valid_inputs = np.array(get_valid_inputs(train_pos, n=1000))

hyperrectangles = []
hyperrectangles_labels = []

# Define "safe region" hyperrectangles around input points
for data in valid_inputs:
    hyperrectangle = []
    for i, d in enumerate(data):
        if i == 14:
            hyperrectangle.append([1e-7, 5e-6])
        elif i == 19:
            hyperrectangle.append([40/1000, 52/1000])
        else:
            hyperrectangle.append([d, d])
    hyperrectangles.append(hyperrectangle)
    hyperrectangles_labels.append(0)

hyperrectangles = np.array(hyperrectangles)
hyperrectangles_labels = np.array(hyperrectangles_labels)

# Load model (need tensorflow==2.15)
model_name = 'adv_mid_bs32_0.999_e8_adv'
model = keras.models.load_model(model_name)


# Initialise PGD
pgd_attack_single_image_loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
adv_acc_metric = keras.metrics.SparseCategoricalAccuracy()

pgd_dataset = []
pgd_labels = []

# Generate an adversarial input by PGD for each safe input, whithin their hyperrectangle 
for i, hyperrectangle in enumerate(hyperrectangles):
    t_hyperrectangle = np.transpose(hyperrectangle)

    # Calculate the epsilon for PGD for each dimension as ((dim[1] - dim[0]) / (pgd_steps * eps_multiplier))
    eps = []
    for d in hyperrectangle:
        eps.append((d[1] - d[0]) / pgd_steps)
    
    # Generate an starting point for PGD by randomly sampling in the hyperrectangle 
    pgd_point = []
    for d in hyperrectangle:
        pgd_point.append(np.random.uniform(d[0], d[1]))
    
    pgd_point = tf.convert_to_tensor([pgd_point], dtype=tf.float32)
    pgd_label = tf.convert_to_tensor([hyperrectangles_labels[i]], dtype=tf.float32)

    # PGD attack
    for _ in range(pgd_steps):
        # Compute PGD loss
        with tf.GradientTape() as tape:
            tape.watch(pgd_point)
            prediction = model(pgd_point, training=False)
            pgd_single_image_loss = pgd_attack_single_image_loss(pgd_label, prediction)
        # Get the gradients of the loss w.r.t to the input point.
        gradient = tape.gradient(pgd_single_image_loss, pgd_point)
        # Get the sign of the gradient to create the perturbation
        signed_grad = tf.sign(gradient)
        # Apply perturbation to image
        pgd_point = pgd_point + signed_grad * eps
        pgd_point = tf.clip_by_value(pgd_point, t_hyperrectangle[0], t_hyperrectangle[1])
        # print(f"PGD step: {pgd_step + 1}", end="\r")

    # Concatenate the adversarial inputs and the original labels
    if len(pgd_dataset) > 0:
        pgd_dataset = np.concatenate((pgd_dataset, pgd_point), axis=0)
        pgd_labels = np.concatenate((pgd_labels, pgd_label), axis=0)
    else:
        pgd_dataset = pgd_point
        pgd_labels = pgd_label

pgd_dataset = np.asarray(pgd_dataset)
pgd_labels = np.asarray(pgd_labels)

# Not all adversarial attacks are successful, so we check each adversarial inputs.
counterexamples_dataset = []
ce_idx = []

for i, point in enumerate(pgd_dataset):
    prediction = model.predict(tf.convert_to_tensor([point]), verbose=0)
    class_index = np.argmax(prediction)
    
    if class_index != pgd_labels[i]:
        print(f"counter-example found for input {i}")
        ce_idx.append(i)
        counterexamples_dataset.append(point)
    
print(ce_idx)
# Print model's accuracy against adversarial attacks and adversarial attacks success rate
adv_acc_2 = 1 - (len(counterexamples_dataset) / len(pgd_dataset))
success_rate = (len(counterexamples_dataset) / len(pgd_dataset))

print(f'Adv acc: {adv_acc_2:.4f}')
print(f'Adv attack uccess rate: {success_rate:.4f}')

# Save the counterexamples to a CSV file
path = 'counterexamples/pgd'
if not os.path.exists(path):
    os.makedirs(path)

np.savetxt(f'{path}/{model_name}.csv', counterexamples_dataset, delimiter=',')

counter-example found for input 139
counter-example found for input 176
counter-example found for input 177
counter-example found for input 325
counter-example found for input 526
counter-example found for input 528
counter-example found for input 621
counter-example found for input 698
counter-example found for input 795
counter-example found for input 865
counter-example found for input 878
counter-example found for input 952
counter-example found for input 957
counter-example found for input 966
[139, 176, 177, 325, 526, 528, 621, 698, 795, 865, 878, 952, 957, 966]
Adv acc: 0.9860
Adv attack uccess rate: 0.0140


In [ ]:
import numpy as np
import re
import os


# Define the regex pattern to match the desired data
pattern = r'x: \[(.*?)\]'

models_directory = './models/tf/SSH'
outputs_directory = './vehicle/outputs'
for output_file in sorted(os.listdir(outputs_directory)):
    output_path = os.path.join(outputs_directory, output_file)
    cube_name = '.'.join(output_file.split('.')[:-1])

    model = keras.models.load_model(os.path.join(models_directory, cube_name))


    # Read the content of the file
    with open(output_path, 'r') as file:
        
        text = file.read()

        # Find all matches
        matches = re.findall(pattern, text)

        # Process each match and convert to a list of lists of floats
        data = []
        for match in matches:
            # Convert the string of numbers into a list of floats
            numbers = [float(num.strip()) for num in match.split(',')]
            data.append(numbers)

        # Convert the list of lists to a NumPy array
        data_array = np.array(data)

        adv_acc = 1 - (len(data_array) / 1000)
        # success_rate = (len(data_array) / 1000)

        print(f'Adv acc: {adv_acc:.4f} for {cube_name}')

        counterexamples_dataset = []

        for i, point in enumerate(data_array):
            prediction = model.predict(tf.convert_to_tensor([point]), verbose=0)
            class_index = np.argmax(prediction)
            
            if class_index != 0:
                counterexamples_dataset.append(point)

        adv_acc_2 = 1 - (len(counterexamples_dataset) / 1000)
        success_rate = (len(counterexamples_dataset) / 1000)

        print(f'Adv acc: {adv_acc_2:.4f} for {cube_name}')
        print(f'Success rate: {success_rate:.4f} for {cube_name}')

        # Save the NumPy array to a CSV file
        path = 'counterexamples/vehicle'
        if not os.path.exists(path):
            os.makedirs(path)
        np.savetxt(f'{path}/{cube_name}.csv', counterexamples_dataset, delimiter=',')
